In [ ]:
!conda install -c huggingface transformers
!pip install transformers
!pip install torch
!pip install chardet
!pip install text2emotion
!pip install emoji==1.6.3
!pip install scrapy

In [ ]:
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy import signals

class AmazonReviewsSpider(scrapy.Spider):
    name = "amazon_reviews"

    def start_requests(self):
        asin_list = ['B00LO33MBG']
        for asin in asin_list:
            amazon_reviews_url = f'https://www.amazon.com/product-reviews/{asin}/'
            yield scrapy.Request(url=amazon_reviews_url, callback=self.parse_reviews, meta={'asin': asin}, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'})

    def parse_reviews(self, response):
        asin = response.meta['asin']

        ## Parse Product Reviews
        review_elements = response.css("#cm_cr-review_list div.review")
        for review_element in review_elements:
            yield {
                "asin": asin,
                "text": "".join(review_element.css("span[data-hook=review-body] ::text").getall()).strip(),
                "title": review_element.css("*[data-hook=review-title]>span::text").get(),
                "rating": review_element.css("*[data-hook*=review-star-rating] ::text").re(r"(\d+\.*\d*) out")[0],
            }

# Run the spider
process = CrawlerProcess(settings={
    'FEED_FORMAT': 'json',
    'FEED_URI': 'output1.json',
    'DOWNLOAD_DELAY': 1,  # Delay of 1 second between requests
})


process.crawl(AmazonReviewsSpider)

In [ ]:
import pandas
from transformers import pipeline
import text2emotion as text2

sentiment_classifier = pipeline("sentiment-analysis")
# Function to classify the sentiment of a given text
def Pipe_sentiment(text):
    result = sentiment_classifier(text)[0]
    # Extract the sentiment and its score
    sentiment = result['label']
    score = result['score']
    return sentiment, score


def emo(text):
  emotions = text2.get_emotion(text)
  angry_score = emotions['Angry']
  sad_score = emotions['Sad']
  fear_score = emotions['Fear']
  happy_score = emotions['Happy']
  surprise_score = emotions['Surprise']

  negative_emotion_score = (angry_score + sad_score + fear_score)/3
  positive_emotion_score = (happy_score + surprise_score)/2

  if negative_emotion_score > positive_emotion_score:
    return "Negative"
  elif negative_emotion_score < positive_emotion_score:
    return "Positive"
  else:
    return "Neutral"

from textblob import TextBlob

def get_sentiment(text):
    blob = TextBlob(text)
    sentiment = blob.sentiment.polarity
    if sentiment > 0:
        return "Positive"
    elif sentiment < 0:
        return "Negative"
    else:
        return "Neutral"




df = pandas.read_json('./output1.json')
for i in range(len(df['text'])):
    text = df['text'][i]
    o = df['rating'][i]
    print(text,o)
    sentiment, score = Pipe_sentiment(text)
    print("Predicted sentiment (Pipe):", sentiment)

    sentiment = get_sentiment(text)
    print("Predicted sentiment (Txtblob):", sentiment)

    sentiment = emo(text)
    print("Predicted sentiment (Text2Emo):", sentiment)